In [39]:
import pandas as pd
import geopandas as gpd
import numpy as np

import sys
sys.path.append('..')
import misc.autointerpolation

# Reload misc module after editing
import importlib
importlib.reload(misc.autointerpolation)
importlib.reload(misc)

# 1. Load and Preprocess Data <a class="anchor" id="load_data"></a>

In [3]:
path = "E:/Projects/SEACAR_WQ_Pilot/"
gis_path = path+'GIS_data/'

# path = "D:/Water_Quailty/"
# gis_path = path+'GIS_data/'

Combine and preprocess discrete and continuous data.

In [4]:
# dfDis, dfCon = misc.preprocess.preprocess(path + r"OEAT_Discrete-2023-Jan-23.csv", 
#                     path + r"OEAT_Continuous_1-2023-Jan-23.csv",
#                     path + r"OEAT_Continuous_2-2023-Jan-23.csv")

# dfAll = pd.concat([dfDis,dfCon]).drop(columns = ['Hour'])

# dfAll.to_csv(path + "all_0214.csv")

In [5]:
# Reload modules in external .py files after editing.
dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])
# Convert SampleDate froms str to date
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate']).dt.date

C:\Users\qiangy\AppData\Local\ESRI\conda\envs\geo3\lib\site-packages\IPython\core\interactiveshell.py:3442: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
col_ls = ['RowID','ParameterName','ParameterUnits','ProgramLocationID','ActivityType','ManagedAreaName',
                   'SampleDate','Year','Month','ResultValue','ValueQualifier','Latitude_DD','Longitude_DD']
para_ls = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
para_ls_ab = ["S","TN","DO","T","SD"]
# Convert full MA names to short names
dictArea    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'GTM Reserve','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}

# Convert full MA names to MA name in ORCP_Managed_Areas_Oct2021
dictArea2    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Gasparilla Sound-Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend Seagrasses',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'Guana Tolomato Matanzas NERR','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island Sound'}

# Convert full MA names to abbreviations
dictArea3    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'ch','Big Bend Seagrasses Aquatic Preserve':'bb',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'gtm','Estero Bay Aquatic Preserve':'eb',
                'Biscayne Bay Aquatic Preserve':'bbay','Matlacha Pass Aquatic Preserve':'mp',
                'Lemon Bay Aquatic Preserve':'lb','Cape Haze Aquatic Preserve':'ch','Pine Island Sound Aquatic Preserve':'pi'}

dictPara = {"Salinity":'S','Total Nitrogen':'TN','Dissolved Oxygen':'DO','Turbidity':'T','Secchi Depth':'SD'}
dictUnits   = {"Salinity":"ppt","Total Nitrogen": "mg/L","Dissolved Oxygen": "mg/L","Turbidity": "NTU", "Secchi Depth": "m"}
listArea    = dfAll["ManagedAreaName"].unique()
listPara    = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
SpatialRef = '3086'

# 2. Run spatial interpolation through seasonality table <a class="anchor" id="combine"></a>

Delete incorrect seasons

In [8]:
dfSeason = pd.read_csv(path + "OEATUSF_Geospatial_TempSeasons_update_0510.csv").drop(columns = "Unnamed: 0")
dfSeason = dfSeason.dropna(subset=["s_start","s_end"]).reset_index(drop=True)

dfSeason['s_start'] = pd.to_datetime(dfSeason['s_start'])
dfSeason["s_end"] = pd.to_datetime(dfSeason["s_end"])
dfSeason["date_range"] = dfSeason["s_end"] - dfSeason["s_start"]
dfSeason["date_range"] = dfSeason.apply(lambda x: int(str(x["date_range"]).split(" ")[0]),axis=1)
dfSeason = dfSeason[dfSeason["date_range"]>0].drop(columns="date_range").reset_index(drop=True)
dfSeason.head()

,Unnamed: 0.1,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,covariates,Processed,Total Nitrogen,Dissolved Oxygen,Salinity,Secchi Depth,Turbidity
0,147,temp,Estero Bay Aquatic Preserve,2022,Summer,7-Jun,2022-06-07,2022-08-26,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
1,71,temp,Biscayne Bay Aquatic Preserve,2022,Summer,1-Jun,2022-06-01,2022-08-26,bathymetry+LDI+popden,1,NaN,NaN,NaN,NaN,NaN
2,70,temp,Biscayne Bay Aquatic Preserve,2022,Spring,2-Mar,2022-03-02,2022-05-31,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
3,146,temp,Estero Bay Aquatic Preserve,2022,Spring,27-Feb,2022-02-27,2022-06-06,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...
4,69,temp,Biscayne Bay Aquatic Preserve,2022,Winter,8-Feb,2022-02-08,2022-03-01,bathymetry+LDI+popden,1,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...,NaN,E:/Projects/SEACAR_WQ_Pilot/GIS_data/output_ra...


In [8]:
# Print all managed areas
display(dfSeason['ma'].unique())

# Create empty columns to store paths of exported files
# data = dfSeason.assign(**{'Total Nitrogen': np.nan, 'Dissolved Oxygen': np.nan, 'Salinity': np.nan, 'Secchi Depth': np.nan, 'Turbidity': np.nan})

array(['Estero Bay Aquatic Preserve', 'Biscayne Bay Aquatic Preserve',
       'Guana Tolomato Matanzas National Estuarine Research Reserve',
       'Gasparilla Sound-Charlotte Harbor Aquatic Preserve',
       'Big Bend Seagrasses Aquatic Preserve'], dtype=object)

In [42]:
for i in dfSeason.index:
# for i in range(176, 211):
    if dfSeason.iloc[i]["Processed"] == 1: continue
    for para in listPara[0:3]:
        name = [dictArea[dfSeason.iloc[i]["ma"]],dfSeason.iloc[i]["st_Year"],dfSeason.iloc[i]["season"],dictPara[para]]
        out_raster,out_ga_layer,ga_to_raster = misc.autointerpolation.interpolation_auto(method = "rk",
                                                                                         gis_path  = gis_path,
                                                                                         dataframe = dfAll,
                                                                                         managed_area = dfSeason.iloc[i]["ma"],
                                                                                         Year = dfSeason.iloc[i]["st_Year"],
                                                                                         Season = dfSeason.iloc[i]["season"],
                                                                                         start_date = dfSeason.iloc[i]["s_start"],
                                                                                         end_date = dfSeason.iloc[i]["s_end"],
                                                                                         parameter = para,
                                                                                         covariates = dfSeason.iloc[i]["covariates"].split("+"),
                                                                                         out_raster = name,
                                                                                         out_ga_layer = name,
                                                                                         predict_std_err = name)
        dfSeason.loc[i, para] = out_raster
        dfSeason.to_csv(path + 'output.csv')
        display('Interpolated Row {}, {}'.format(i,name))

Start the interpolation with the RK method
Salinity in 2021 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).



"Interpolated Row 6, ['Estero Bay', 2021, 'Fall', 'S']"

Start the interpolation with the RK method
--- Time lapse: 120.7061095237732 seconds ---


"Interpolated Row 6, ['Estero Bay', 2021, 'Fall', 'TN']"

Start the interpolation with the RK method
--- Time lapse: 118.93717074394226 seconds ---


"Interpolated Row 6, ['Estero Bay', 2021, 'Fall', 'DO']"

Start the interpolation with the RK method


KeyboardInterrupt: 